# Entrega laboratorio 2

## Integrantes 
-
-

In [1]:
from pyomo.environ import ConcreteModel, RangeSet, \
    Param, Var, Binary, \
    Objective, maximize, Constraint, value, \
    NonNegativeIntegers, minimize, \
    NonNegativeReals

import numpy
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from typing import List

from pyomo.opt import SolverFactory

# Punto 1

El objetivo es transportar recursos esenciales a diferentes zonas de Zambia utilizando una flota de 3 aviones. 

## Sets
- $F = F_1, F_2, F_3$ conjunto de los 3 aviones
- $R = R_1, \dots , R_5$ conjunto de los 5 recursos  

## Parámetros
- $W_i$ peso máximo que puede transportar el $i \in F$ avión
- $V_i$ volumen máximo que puede transportar el $i \in F$ avión
- $w_j$ peso del $j \in R$ recurso
- $v_j$ volumen del $j \in R$ recurso
- $p_j$ beneficio de llevar el $j \in R$ recurso

## Variables de decisión
- $X_{i,j}$ matriz de reales no negativos que indica cuanto se lleva del recurso $j \in R$ en el avión $i \in F$ en kilogramos.
- $Z_{i,j}$ matriz binaria que indica si el $i \in F$ avión llevan el $j \in R$ recurso. 

## Función objetivo
Se quiere maximizar el valor de los recursos que se llevan en la flota de aviones.
$$
\max \sum_{i \in F} \sum_{j \in R} X_{i,j} \times \frac{p_j}{m_j}
$$

## Restricciones
1. Cada uno de los aviones no puede llevar mas peso que su capacidad maxima.
$$
\sum_{j \in R} X_{i,j} \leq W_i, \forall i \in F
$$
2. Cada uno de los aviones no puede llevar mas volumen que su capacidad maxima.
$$
\sum_{j \in R} X_{i,j} \times v_j \leq V_i, \forall i \in F
$$
3. No se puede llevar una cantidad negativa de recursos.
$$
X_{i,j} \geq 0, \forall i \in F \,\,\, \forall j \in R
$$
4. No se pueden llevar mas recursos que los disponibles. Es decir, lo que se puede llevar en un avión no debe exceder la disponibilidad del recurso mismo
$$
\sum_{i \in F} X_{i,j} \leq m_j , \, \forall j \in R
$$
5. Los equipos medicos se tienen que llevar en unidades de 300 kg.
$$
X_{i,3} \mod 300 = 0
$$
6. (Logistica 1) Las medicinas no pueden ir en el avión 1.
$$
X_{1,2} = 0
$$
7. (Logistica 2) El agua y los equipos medicos no pueden ir juntos en un avión para evitar contaminación cruzada. En este punto se puede optar por definir una variable auxiliar $Z$, que será una matriz binaria (0 y 1), de las dimensiones de $X$ tal que se almacena en la entrada $X_{i,j}$ si se lleva o no un recurso. De tal manera que se pueda realizar la restricción de manera más sencilla. 
$$
Z_{i,3} + Z_{i,4} \leq 1
$$

Al introducir esta variable auxiliar es necesario relacionarla con la variable objetivo. Por tanto, nacen dos restricciones mas: 1) Si $X_{i,j} > 0$, entonces $Z_{i,j} = 0$. 2) Si $X_{i,j} = 0$, entonces $Z_{i,j} = 0$.

## Tipo de problema 

In [14]:
class AsignationProblemSolver:
    """
    A class to solve the fractional problem humanitarian mission.
    ...
    Attributes
    ----------
    """

    def __init__(self, profit, w_res, v_res, maxw_fli, maxv_fli):
        flights_num = len(profit)
        if flights_num != len(w_res):
            raise "Error: cada recurso debe tener un peso"
        elif flights_num != len(v_res):
            raise "Error: cada recurso debe tener un volumen"

        if len(maxv_fli) != len(maxw_fli):
            raise "Error: todos los aviones tienen un volumen y un peso maximo para transportar"

        self.dprofit = {i+1: profit[i] for i in range(len(profit))}
        self.dw_res = {i+1: w_res[i]*1000 for i in range(len(w_res))} # kg
        self.dv_res = {i+1: v_res[i] for i in range(len(v_res))} # m^3 / kg
        
        self.dmaxw_fli = {i+1: maxw_fli[i]*1000 for i in range(len(maxw_fli))} # kg
        self.dmaxv_fli = {i+1: maxv_fli[i] for i in range(len(maxv_fli))} # m^3

        self.num_res = len(profit)
        self.num_fli = len(maxw_fli)

    def setup(self):
        model = ConcreteModel()

        # sets
        model.R = RangeSet(1, self.num_res)
        model.F = RangeSet(1, self.num_fli)

        # params
        model.profit = Param(model.R, initialize=self.dprofit)
        model.w_res = Param(model.R, initialize=self.dw_res) # kg
        model.v_res = Param(model.R, initialize=self.dv_res) # m^3 / kg

        model.w_fli = Param(model.F, initialize=self.dmaxw_fli) # kg
        model.v_fli = Param(model.F, initialize=self.dmaxv_fli) # m^3

        # aux
        # relation between Z / F {1, 0}
        model.Z = Var(model.F, model.R, within=Binary)
        # decision variable peso usado
        model.X = Var(model.F, model.R, within=NonNegativeReals)

        # objective function
        def obj_function(model):
            return sum(model.X[i,j] * model.profit[j] / model.w_res[j] for j in model.R for i in model.F)
        model.obj_function = Objective(rule=obj_function, sense=maximize)

        # constraints
        # relacionar X y W
        model.relation1 = Constraint(model.F, model.R, expr=lambda model, i, j: model.X[i,j] <= 10**6 * model.Z[i,j])
        
        def volume_constr(model, i):
            return sum(model.X[i,j]*model.v_res[j] for j in model.R) <= model.v_fli[i]
        model.volume_constr = Constraint(model.F, rule=volume_constr)

        def weight_constr(model, i):
            return sum(model.X[i,j] * model.w_res[j] for j in model.R) <= model.w_fli[i]
        model.weight_constr = Constraint(model.F, rule=weight_constr)

        # para cada recurso que se lleva en un avion no se puede llevar mas de una unidad
        # def res_constr(model, j):
        #     return sum(model.X[i, j] for i in model.F) <= 1
        # model.res_constr = Constraint(model.R, rule=res_constr)

        # def log_constr1(model, i):
        #     return model.X[i,4] >= 99 * model.Z[i,3]
        # model.log_constr1 = Constraint(model.F, rule=log_constr1)
        # def log_constr2(model, i):
        #     return model.X[i,3] >= 99 * model.Z[i,4]
        # model.log_constr2 = Constraint(model.F, rule=log_constr2)

        model.medications = Constraint(expr=lambda model: model.X[1,2] == 0)
        model.constr = Constraint(model.F, expr=lambda model, i: model.Z[i,3] + model.Z[i, 4] <= 1)
        model.r = Constraint(model.F, expr=lambda model, i: (model.X[i,3] / 300) == 0)

        self.model = model

    def solve(self):
        solver = SolverFactory('glpk')
        solver.solve(self.model)

    def print_solution(self):
        for i in self.model.F:
            print(f'En el avion {i} van los recursos:')
            w, v, p = 0, 0, 0
            for j in self.model.R:
                if self.model.X[i,j].value > 0:
                    wei = self.model.w_res[j] * self.model.X[i, j].value
                    vol = self.model.v_res[j] * self.model.X[i, j].value 
                    pro = self.model.profit[j] * self.model.X[i, j].value
                    
                    print(f'\t[->] El recurso {j} con beneficio {pro:.2f}')
                    print(f'\t\t Volumen: {vol:.2f} m^3')
                    print(f'\t\t Peso: {wei:.2f} kg')
                    
                    w += wei
                    v += vol
                    p += pro
                
            print('\t---- STATS ----')
            print(f'\tPeso usado: {w:.2f}/{self.model.w_fli[i]} kg')
            print(f'\tVolumen usado: {v:.2f}/{self.model.v_fli[i]} m^3')
            print(f'\tBeneficio obtenido: {p:.2f}')
            print()

        print('*'*20, end='\n'*2)
        for i in self.model.F:
            for j in self.model.R:
                print(f"({self.model.X[i,j].value}, {self.model.Z[i,j].value})", end="")
            print()

        print(f'\nProfit: {sum([self.model.profit[j] * self.model.X[i,j].value for j in self.model.R for i in self.model.F])}')
                

In [15]:
profit = [50, 100, 120, 60, 40]
w_res = [15, 5, 20, 18, 10]
v_res = [8, 2, 10, 12, 6]

maxw_fli = [30, 40, 50]
maxv_fli = [25, 30, 35]

solver = AsignationProblemSolver(profit, w_res, v_res, maxw_fli, maxv_fli)
solver.setup()
solver.solve()
solver.print_solution()

En el avion 1 van los recursos:
	[->] El recurso 5 con beneficio 120.00
		 Volumen: 18.00 m^3
		 Peso: 30000.00 kg
	---- STATS ----
	Peso usado: 30000.00/30000 kg
	Volumen usado: 18.00/25 m^3
	Beneficio obtenido: 120.00

En el avion 2 van los recursos:
	[->] El recurso 2 con beneficio 800.00
		 Volumen: 16.00 m^3
		 Peso: 40000.00 kg
	---- STATS ----
	Peso usado: 40000.00/40000 kg
	Volumen usado: 16.00/30 m^3
	Beneficio obtenido: 800.00

En el avion 3 van los recursos:
	[->] El recurso 2 con beneficio 1000.00
		 Volumen: 20.00 m^3
		 Peso: 50000.00 kg
	---- STATS ----
	Peso usado: 50000.00/50000 kg
	Volumen usado: 20.00/35 m^3
	Beneficio obtenido: 1000.00

********************

(0.0, 0.0)(0.0, 0.0)(0.0, 0.0)(0.0, 0.0)(3.0, 1.0)
(0.0, 0.0)(8.0, 1.0)(0.0, 0.0)(0.0, 0.0)(0.0, 0.0)
(0.0, 0.0)(10.0, 1.0)(0.0, 0.0)(0.0, 0.0)(0.0, 0.0)

Profit: 1920.0


# Punto 2

In [4]:
class TransportProblemSolver:
    """
    A class to solve the transport problem.
    ...
    Attributes
    ----------
    costs : List[List[float]]
        A matrix in which each entry represents the cost of going from a distribution
        city to a destination city.
    offer : List[int]
        An array with the availability of resources in each distribution city.
    demand : List[int]
        An array with the demand of resources in each destination city.
    """

    def __init__(self, costs: List[List[float]], offer: List[int], demand: List[int]):
        if len(costs) == 0:
            raise "Error: la matriz de costos no puede ser vacia"
        elif len(demand) != len(costs):
            raise "Error: cada j ciudad destino debe tener una fila en la matriz"
        elif len(offer) != len(costs[0]):
            raise "Error: cada i ciudad distribuidora debe tener una columna en la matriz"

        self.offer = offer
        self.costs = costs
        self.demand = demand

    def setup(self):
        model = ConcreteModel()

        # sets
        model.D = RangeSet(1, len(self.demand)) # destination
        model.O = RangeSet(1, len(self.offer)) # origin

        # params
        dict_ofert = {i+1: self.offer[i] for i in range(len(self.offer))}
        dict_demand = {i+1: self.demand[i] for i in range(len(self.demand))}
        dict_costs = {
            (i+1, j+1): self.costs[i][j]
            for j in range(len(self.costs[0])) for i in range(len(self.costs))
        }

        model.offer = Param(model.O, initialize=dict_ofert)
        model.demand = Param(model.D, initialize=dict_demand)
        model.costs = Param(model.D, model.O, initialize=dict_costs)
        
        # dec. variable
        model.X = Var(model.D, model.O, within=NonNegativeIntegers)

        # objective function
        def obj_function(model):
            return sum(model.X[i, j] * model.costs[i, j] for j in model.O for i in model.D)
        model.obj_function = Objective(rule=obj_function, sense=minimize)

        # constraints
        def offer_constr(model, j):
            return sum(model.X[i,j] for i in model.D) <= model.offer[j]
        model.offer_contr = Constraint(model.O, rule=offer_constr)

        def demand_constr(model, i):
            return sum(model.X[i,j] for j in model.O) >= model.demand[i]
        model.demand_constr = Constraint(model.D, rule=demand_constr)

        self.model = model

    def solve(self):
        solver = SolverFactory('glpk')
        solver.solve(self.model)

    def print_solution(self):
        for j in self.model.O:
            print(f"Desde {j} ciudad se ha sumistrado recursos hacia: ")
            ofert = 0
            for i in self.model.D:
                if self.model.X[i, j].value != 0:
                    ofert += self.model.X[i,j].value
                    resources = self.model.X[i,j].value
                    print(f"\t[->] destino {i} un total de {resources} recursos (min: {self.model.demand[i]})")
            print('\t---- STATS ----')
            print(f'\tused: {ofert}/{self.model.offer[j]} from distribution city\n')

In [5]:
offer = [550, 700]
costs = [
    [9999, 2.5],
    [2.5, 9999],
    [1.6, 2.0],
    [1.5, 1.0],
    [0.8, 1.0],
    [1.4, 0.8]
]
demand = [125, 175, 225, 250, 225, 200]

solver = TransportProblemSolver(costs, offer, demand)
solver.setup()
solver.solve()
solver.print_solution()

Desde 1 ciudad se ha sumistrado recursos hacia: 
	[->] destino 2 un total de 175.0 recursos (min: 175)
	[->] destino 3 un total de 225.0 recursos (min: 225)
	[->] destino 5 un total de 150.0 recursos (min: 225)
	---- STATS ----
	used: 550.0/550 from distribution city

Desde 2 ciudad se ha sumistrado recursos hacia: 
	[->] destino 1 un total de 125.0 recursos (min: 125)
	[->] destino 4 un total de 250.0 recursos (min: 250)
	[->] destino 5 un total de 75.0 recursos (min: 225)
	[->] destino 6 un total de 200.0 recursos (min: 200)
	---- STATS ----
	used: 650.0/700 from distribution city



# Punto 3

## Conjuntos
Se plantean los siguientes dos conjuntos.

- $F = \{1, 2, \dots, 8\}$ conjunto con las filas del tablero
- $C = \{1, 2, \dots, 8\}$ conjunto con las columnas del tablero

## Variable objetivo
Se define $X_{ij}$ como una matriz binaria de tamaño $8\times 8$ con $i \in F$ y $j \in C$. Esta matriz $X_{ij}$ será será 1 si hay una reina en esa posición y 0 de lo contrario.

## Función objetivo 
Se quiere minimizar el número de reinas que se ubican en el tablero de ajedrez. Por tanto,

$$
\min \sum_{i \in F} \sum_{j \in C} X_{ij}
$$

## Restricciones
- Cada casilla tiene que estar cubierta por lo menos por una reina.

$$
\sum_{k \in F} X_{k, j} + \sum_{k \in C} X_{i, k} + \sum_{i- k, j-k}^{8} + \sum_{i+ k, j+k}^{8} + \sum_{i- k, j+k}^{8} + \sum_{i+k, j-k}^{8} \geq 1 \,, \forall i \in F, j \in C \, | \, 1 \leq i \leq 8 \wedge 1 \leq j \leq 8
$$

In [ ]:
from pyomo.environ import *

# Definir el modelo
model = ConcreteModel()

# Conjuntos
n = 8  # Tamaño del tablero
model.F = RangeSet(1, n)
model.C = RangeSet(1, n)

# Variables de decisión
model.X = Var(model.F, model.C, within=Binary)  # 1 si hay una reina en (i, j)

# Función objetivo: minimizar el número de reinas
model.obj = Objective(expr=sum(model.X[i, j] for i in model.F for j in model.C), sense=minimize)

# Restricción de cobertura: cada casilla debe estar cubierta por al menos una reina
def cover_rule(model, i, j):
    return sum(model.X[k, j] for k in model.F) + \
           sum(model.X[i, k] for k in model.C) + \
           sum(model.X[i-k, j-k] for k in range(1, n) if 1 <= i-k <= n and 1 <= j-k <= n) + \
           sum(model.X[i+k, j+k] for k in range(1, n) if 1 <= i+k <= n and 1 <= j+k <= n) + \
           sum(model.X[i-k, j+k] for k in range(1, n) if 1 <= i-k <= n and 1 <= j+k <= n) + \
           sum(model.X[i+k, j-k] for k in range(1, n) if 1 <= i+k <= n and 1 <= j-k <= n) >= 1

model.cover_constraints = Constraint(model.F, model.C, rule=cover_rule)

# Resolver el modelo
solver = SolverFactory('glpk')
solver.solve(model)

In [36]:
print("="*10 + " tablero " + "="*10)
for i in model.F:
    row = "".join("Q " if model.X[i, j].value == 1 else ". " for j in model.C)
    print(row)

mapper = {1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h'}

l = [f"{mapper[j]}{9 - i}" for i in model.F for j in model.C if model.X[i,j].value == 1]
print()
print("="*10 + "selected queens" + "="*10)
print(l)

========== tablero ==========
Q . . . . . . . 
. . . . Q . . . 
. . . . . . Q . 
. . . . . . . . 
. . . . . . . . 
. . . . . . . . 
. Q . . . . . . 
. . . . . Q . . 

==========selected queens==========
['a8', 'e7', 'g6', 'b2', 'f1']


In [ ]:
"""
This script solves the minimum queen cover problem using Pyomo in a class-based structure.
Description: 
Given a chessboard of size 8 x 8, the goal is to place the minimum number of queens such that every
square is attacked by at least one queen.

Created on Mon March 25 17:07:12 2024

@author:
    Juan Andrés Méndez Galvis
"""

import time
from pyomo.environ import (
    Binary,
    ConcreteModel,
    ConstraintList,
    Objective,
    RangeSet,
    Var,
)
from pyomo.opt import SolverFactory
from queen_mapper import visualize_queens


class QueenCoverSolver:
    """
    Class to solve the minimum queen cover problem using Pyomo.

    Given a chessboard of a specified size (default 8 x 8), the goal is to place
    the minimum number of queens such that every square is attacked by at least one queen.
    """

    def __init__(self, board_size=8):
        """
        Initialize the solver with a given board size.

        Args:
            board_size (int): Size of the chessboard (default is 8).
        """
        self.board_size = board_size
        self.coverage_matrix = {}
        self.model = None

    def generate_coverage_matrix(self):
        """
        Calculates which squares each queen can attack on a chessboard.

        Returns:
            dict: A dictionary representing the coverage matrix. Keys are (square1, square2)
                  tuples, and values are 1 if a queen placed on square2 can attack square1, 0 otherwise.
        """
        coverage_matrix = {}
        # TODO: Implement the algorithm to compute the coverage matrix.
        # Hint: Queens can attack horizontally, vertically, and diagonally.

        
        
        self.coverage_matrix = coverage_matrix
        return coverage_matrix

    def create_pyomo_model(self, coverage_matrix):
        """
        Creates the Pyomo model for the queen placement problem.

        Args:
            coverage_matrix (dict): The coverage matrix computed for the chessboard.

        Returns:
            ConcreteModel: A Pyomo ConcreteModel representing the problem.
        """
        model = ConcreteModel()

        # TODO: Implement the Pyomo model for the queen placement problem.
        # Define the set of squares on the board.
        # Hint: Remeber not because the board is essentially a 2D grid means that your set size should behave like a 2D grid.

        # Define binary decision variables.

        # Define the objective function: minimize the total number of queens placed.

        # Define constraints

        self.model = model
        return model

    def solve_and_visualize(self):
        """
        Solves the Pyomo model and visualizes the queen placements.

        This method should:
         - Solve the model using a specified solver (e.g., 'glpk').
         - Print the objective function value.
         - Extract the solution and visualize the queen placements using the
           'visualize_queens' function.
        """
        if self.model is None:
            raise ValueError("The model has not been created. Run create_pyomo_model() first.")

        solver = SolverFactory("glpk")
        results = solver.solve(self.model, tee=True)

        print("Objective function value: ", self.model.obj())
        print("Results:", results)

        queens = []
        for i in self.model.squares:
            if self.model.x[i]() == 1:
                pos = (i - 1) % 8, (i - 1) // 8  # Convert to chess notation
                queens.append(chr(pos[0] + 97) + str(pos[1] + 1))

        visualize_queens(queens)

In [ ]:
solver = QueenCoverSolver(board_size=8)
start_time = time.time()
coverage_matrix = solver.generate_coverage_matrix()
print("Coverage matrix computed in:", time.time() - start_time, "seconds")

model = solver.create_pyomo_model(coverage_matrix)

solver.solve_and_visualize()

# Punto 4

# Punto 5